## Run the code using In-hospital network

In [ ]:
import vitaldb
import pandas as pd
import pymysql
import os

def create_safe_tuple_string(items):
    """Create a safe SQL IN clause string from a list of items."""
    items = [int(item) for item in items]
    if len(items) == 1:
        return f"({items[0]})"  # Single item
    else:
        return str(tuple(items))  # Multiple items

# Database connection
try:
    db = pymysql.connect(
        host='172.16.142.70',
        port=3306,
        user='vitaldb',
        passwd='qkdlxkf2469',
        db='snuop',
        charset='utf8'
    )
    cur = db.cursor()

    # Define target tracks
    target_tracks = ('Root/EEG_L','Root/EEG_L1','Root/EEG_L2', 'Root/EEG_R','Root/EEG_R1','Root/EEG_R2')
    
    # Get track IDs
    trkname_sql = f"SELECT * FROM tracks WHERE tracks.trkname IN {str(target_tracks)};"
    cur.execute(trkname_sql)
    data = cur.fetchall()
    trkdf = pd.DataFrame(data, columns=['trkid', 'trkname'])
    track_ids = trkdf['trkid'].values
    trkname = list(trkdf['trkname'])

    # Construct and execute the file list query
    track_tuple_str = create_safe_tuple_string(track_ids)
    filelst_sql = f"""
        SELECT vitalfiles.filename, vitalfiles.dtstart, vitalfiles.dtend, vitalfiles_trk.trkid 
        FROM vitalfiles 
        INNER JOIN vitalfiles_trk ON vitalfiles_trk.fileid = vitalfiles.fileid 
        WHERE vitalfiles_trk.trkid IN {track_tuple_str}
    """
    cur.execute(filelst_sql)
    
    data = cur.fetchall()
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=['filename', 'dtstart', 'dtend', 'trkid'])
    
    df.to_csv('eeg_list.csv')
    print(f"Successfully retrieved {len(df)} records")
    
except pymysql.Error as e:
    print(f"Database error occurred: {e}")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    if 'db' in locals():
        db.close()
'''
df = pd.read_csv('./eeg_list.csv')
df['dtstart'] = pd.to_datetime(df['dtstart'])
df['dtstart'].min(), df['dtstart'].max()
'''
# CSV 파일 읽기
df = pd.read_csv('./eeg_list.csv')
df = df[~df['filename'].str.contains('CU', na=False)]
df = df[~df['filename'].str.contains('P', na=False)]
df['dtstart'] = pd.to_datetime(df['dtstart'])

# VitalDB 로그인
if vitaldb.login('eundain94', 'eundain94', '172.16.142.27', '80'):
    DOWNLOAD_DIR = 'D:/data'
    
    # 디렉토리가 없으면 생성
    if not os.path.exists(DOWNLOAD_DIR):
        os.makedirs(DOWNLOAD_DIR)
    
    # DataFrame의 각 파일에 대해 다운로드 수행
    for filename in df['filename'].unique():  # filename 열의 이름이 'filename'이라고 가정
        opath = os.path.join(DOWNLOAD_DIR, filename)
        try:
            print(f"Downloading {filename}...")
            vitaldb.download(filename, opath)
            print(f"Successfully downloaded {filename}")
        except Exception as e:
            print(f"Failed to download {filename}: {e}")
            continue
    print("Download process completed")
else:
    print("Failed to login to VitalDB")

## Possible to use any network

In [1]:
import os
import pandas as pd
import os, shutil
import numpy as np
from tqdm import tqdm
import re
from sklearn.model_selection import train_test_split
import pickle


In [8]:
vital_files =[]
for file in os.listdir("D:/data"):
    if file.endswith(".vital"):
        vital_files.append(file[:-6])

x1 = pd.read_csv('adt.csv', usecols=['hid1', 'filename']) # downloaded from VitalDB ADT
x1['hid1'] = x1['hid1'].astype(str).apply(lambda x: '0'+x if len(x)==7 else x)
x1 = x1[x1['hid1'].str.match(r'^\d{8}$')]
x1= x1[x1['filename'].isin(vital_files)]

x2= pd.read_csv('eeg_list.csv')
dict_start = dict(zip(x2['filename'].str.replace('.vital',''),x2['dtstart']))
dict_end = dict(zip(x2['filename'].str.replace('.vital',''),x2['dtend']))

x1['dtstart'] = x1['filename'].map(dict_start)
x1['dtend']=x1['filename'].map(dict_end)
x1 = x1.rename(columns={'hid1':'hid'})
x1 = x1[['hid','filename','dtstart','dtend']]
x1.to_csv('eeg_hid.csv', index=False)
# Make eeg_hid.xlsx from eeg_hid.csv, then upload to RidEx to get eeg_rid.csv
# Leave only rid column (without header), remove blank rows, and upload to Supreme in '환자등록번호' section
# Then download delirium.csv from Supreme

C:\Users\안장호\AppData\Local\Temp\ipykernel_23476\2844804582.py:6: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  x1 = pd.read_csv('adt.csv', usecols=['hid1', 'filename']) # downloaded from VitalDB ADT


In [5]:
x1 = pd.read_csv('delirium_old.csv', encoding='cp949', encoding_errors='ignore')
x2 = pd.read_csv('eeg_rid.csv')

x1['생년월일']= pd.to_datetime(x1['생년월일'])
x1['입원일자']= pd.to_datetime(x1['입원일자'])
x1['퇴원일자']= pd.to_datetime(x1['퇴원일자'])
x1['약품처방일']= pd.to_datetime(x1['약품처방일'])
x1['서식작성일']= pd.to_datetime(x1['서식작성일'])
x2['dtstart'] = pd.to_datetime(pd.to_datetime(x2['dtstart']).dt.date)

x1['label_c']=0
x2['label_p']=0
x2['exclude_p']=0
x2['label_c']=0
x2['exclude_c']=0

for idx, row in tqdm(x2.iterrows(), total=len(x2), desc="Processing prescriptions"):
    matching_rows = x1[x1['연구별 환자 ID']==row['hid']]
    has_prescription = matching_rows[
        #(matching_rows['약품명(성분명)'].str.contains('haloperidol', case=False, na=False)) &
        (matching_rows['입원일자']<=row['dtstart']) &        
        (row['dtstart']<=matching_rows['약품처방일']) &
        (matching_rows['약품처방일']<=matching_rows['퇴원일자']) 
        #(matching_rows['약품처방일']<=(row['dtstart']+pd.DateOffset(days=7)))
    ].shape[0] >0
    x2.loc[idx, 'label_p']=1 if has_prescription else 0
    if has_prescription:
        # Find minimum date among only positive prescription dates
        positive_dates = matching_rows[matching_rows['약품처방일'] > row['dtstart']]['약품처방일']
        if not positive_dates.empty:
            x2.loc[idx,'label_p_interval'] = int((positive_dates.min() - row['dtstart']).days)
print(x2['label_p'].value_counts())

for idx, row in tqdm(x2.iterrows(), total=len(x2), desc="Checking prior prescriptions"):
    matching_rows = x1[x1['연구별 환자 ID']==row['hid']]
    has_prescription_before = matching_rows[
        #(matching_rows['약품명(성분명)'].str.contains('haloperidol', case=False, na=False)) &
        (matching_rows['입원일자']<=matching_rows['약품처방일']) &
        (matching_rows['약품처방일']<=row['dtstart']) &
        (row['dtstart']<=matching_rows['퇴원일자'])
    ].shape[0] >0
    if has_prescription_before:
        x2.loc[idx, 'exclude_p']=1
print(x2['exclude_p'].value_counts())

patterns = [
    r'A\>.*?delirium.*?P\>',
    r'A\>.*?delirium.*?Rec\>',
    r'Ass\>.*?delirium.*?Rec\>'
]
x1['label_c'] = x1['서식내용'].fillna('').str.contains(
    '|'.join(patterns), 
    case=False, 
    regex=True,
    flags=re.DOTALL  # Allow matching across newlines
).astype(int)
for idx, row in tqdm(x2.iterrows(), total=len(x2), desc="Processing CSTs"):
    matching_rows = x1[x1['연구별 환자 ID']==row['hid']]
    has_prescription = matching_rows[
        (matching_rows['label_c']==1) &
        (matching_rows['입원일자']<=row['dtstart']) &        
        (row['dtstart']<=matching_rows['서식작성일']) &
        (matching_rows['서식작성일']<=matching_rows['퇴원일자'])
        #(matching_rows['서식작성일']<=(row['dtstart']+pd.DateOffset(days=7)))
    ].shape[0] >0
    x2.loc[idx, 'label_c']=1 if has_prescription else 0
    if has_prescription:
        positive_dates = matching_rows[matching_rows['서식작성일'] > row['dtstart']]['서식작성일']
        if not positive_dates.empty:
            x2.loc[idx,'label_c_interval'] = int((positive_dates.min() - row['dtstart']).days)
print(x2['label_c'].value_counts())

for idx, row in tqdm(x2.iterrows(), total=len(x2), desc="Checking prior CSTs"):
    matching_rows = x1[x1['연구별 환자 ID']==row['hid']]
    has_prescription_before = matching_rows[
        (matching_rows['label_c']==1) &
        #(matching_rows['약품명(성분명)'].str.contains('haloperidol', case=False, na=False)) &
        (matching_rows['입원일자']<=matching_rows['서식작성일']) &
        (matching_rows['서식작성일']<=row['dtstart']) &
        (row['dtstart']<=matching_rows['퇴원일자'])
    ].shape[0] >0
    if has_prescription_before:
        x2.loc[idx, 'exclude_c']=1
print(x2['exclude_c'].value_counts())
x2.to_csv('eeg_rid_supreme_labelled_raw.csv', index=False, encoding = 'utf-8-sig')

x3 = x2[(x2['exclude_p']==0)&(x2['exclude_c']==0)]
x3 = x3.drop(['exclude_p','exclude_c'],axis=1)

birth_date_map = x1.set_index('연구별 환자 ID')['생년월일'].to_dict()
x3['생년월일'] = x3['hid'].map(birth_date_map)
x3['age'] = (x3['dtstart']-x3['생년월일']).dt.days/365.25
x3 = x3[x3['age']>=18]

# Create a combined label column
# If either label_p or label_c is 1, set label to 1, otherwise 0
x3['label'] = ((x3['label_p'] == 1) | (x3['label_c'] == 1)).astype(int)
# Check the distribution of the new label column

print(x3['label'].value_counts())
x3.to_csv('eeg_rid_supreme_labelled.csv', index=False, encoding = 'utf-8-sig')

C:\Users\oskum\AppData\Local\Temp\ipykernel_24348\2300523585.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  x2['dtstart'] = pd.to_datetime(pd.to_datetime(x2['dtstart']).dt.date)
Processing prescriptions: 100%|██████████| 35115/35115 [08:06<00:00, 72.23it/s]


label_p
0    34675
1      440
Name: count, dtype: int64


Checking prior prescriptions: 100%|██████████| 35115/35115 [08:05<00:00, 72.31it/s]


exclude_p
0    34906
1      209
Name: count, dtype: int64


Processing CSTs: 100%|██████████| 35115/35115 [07:06<00:00, 82.31it/s]


label_c
0    34987
1      128
Name: count, dtype: int64


Checking prior CSTs: 100%|██████████| 35115/35115 [07:01<00:00, 83.37it/s]


exclude_c
0    35055
1       60
Name: count, dtype: int64
label
0    34283
1      267
Name: count, dtype: int64


In [2]:
x3 = pd.read_csv('eeg_rid_supreme_labelled.csv')
# Get unique HIDs and their corresponding filenames
hid_to_files = x3.groupby('hid')['filename'].unique().to_dict()
# Get unique HIDs
unique_hids = list(hid_to_files.keys())
# Split HIDs into train/test
train_hids, test_hids = train_test_split(unique_hids, test_size=0.1, random_state=1)
# Get filenames for train and test sets
train_files = []
for hid in train_hids:
    train_files.extend(hid_to_files[hid])
train_files = list(set(train_files))  # Remove any duplicates
test_files = []
for hid in test_hids:
    test_files.extend(hid_to_files[hid])
test_files = list(set(test_files))  # Remove any duplicates

# Save train and test filename lists
with open('train_files.pkl', 'wb') as f:
    pickle.dump(train_files, f)
with open('test_files.pkl', 'wb') as f:
    pickle.dump(test_files, f)

print(f"Number of train files: {len(train_files)}")
print(f"Number of test files: {len(test_files)}")

# Check label distribution in train and test sets
train_labels = x3[x3['filename'].isin(train_files)]['label'].value_counts()
test_labels = x3[x3['filename'].isin(test_files)]['label'].value_counts()

print("\nTrain set label distribution:")
print(train_labels)
print(f"Positive ratio: {train_labels[1]/(train_labels[0] + train_labels[1]):.3f}")

print("\nTest set label distribution:") 
print(test_labels)
print(f"Positive ratio: {test_labels[1]/(test_labels[0] + test_labels[1]):.3f}")

Number of train files: 31043
Number of test files: 3507

Train set label distribution:
label
0    30802
1      241
Name: count, dtype: int64
Positive ratio: 0.008

Test set label distribution:
label
0    3481
1      26
Name: count, dtype: int64
Positive ratio: 0.007


In [3]:
x3 = pd.read_csv('eeg_rid_supreme_labelled.csv')
with open('train_files.pkl', 'rb') as f:
    train_files = pickle.load(f)
with open('test_files.pkl', 'rb') as f:
    test_files = pickle.load(f)

test_data = x3[x3['filename'].isin(test_files)]
test_pos = test_data[test_data['label'] == 1]
test_neg = test_data[test_data['label'] == 0]

os.makedirs('D:/test', exist_ok=True)
test = x3.loc[x3['filename'].isin(test_files), 'filename'].unique()
for filename in test:
    src = os.path.join('D:/data',f'{filename}.vital')
    dst = os.path.join('D:/test', f'{filename}.vital')
    if os.path.exists(src) and os.path.getsize(src) > 0:
        shutil.copy2(src,dst)
#####################################################################################
train_data = x3[x3['filename'].isin(train_files)]
# Separate into positive and negative samples
train_pos = train_data[train_data['label'] == 1]
train_neg = train_data[train_data['label'] == 0]

n_pos = len(train_pos['filename'].unique())
n_neg = 5*n_pos  # We want twice as many negatives as positives
# Randomly sample negative cases
train_neg_sampled = train_neg.drop_duplicates('filename').sample(n=n_neg, random_state=42)
# Combine positive and sampled negative cases
train_5 = pd.concat([train_pos, train_neg_sampled])
train_5 = train_5['filename'].unique()
os.makedirs('D:/train_5', exist_ok=True)
for filename in train_5:
    src = os.path.join('D:/data',f'{filename}.vital')
    dst = os.path.join('D:/train_5', f'{filename}.vital')
    if os.path.exists(src) and os.path.getsize(src) > 0:
        shutil.copy2(src,dst)
########################################################################################
x3[x3['filename'].isin(test)].to_csv('eeg_rid_supreme_test.csv', index=False)
x3[x3['filename'].isin(train_5)].to_csv('eeg_rid_supreme_train_5.csv', index=False)

In [ ]:
test_data = pd.read_csv('eeg_rid_supreme_test.csv')
train_data = pd.read_csv('eeg_rid_supreme_train_5.csv')
eeg_rid_supreme_sampled = pd.concat([test_data, train_data], ignore_index=True)
eeg_rid_supreme_sampled.to_csv('eeg_rid_supreme_sampled.csv', index=False)

In [25]:
eeg_rid_supreme_test = pd.read_csv('eeg_rid_supreme_test.csv')

hid_grouped = eeg_rid_supreme_test.groupby('hid')['label'].agg(lambda x: x.value_counts().index[0]).reset_index()
eeg_rid_supreme_test_unique_hid = eeg_rid_supreme_test.drop_duplicates('hid')
eeg_rid_supreme_test_unique_hid = eeg_rid_supreme_test_unique_hid.merge(hid_grouped, on='hid', how='left', suffixes=('_orig', ''))

positive_hids = eeg_rid_supreme_test_unique_hid[eeg_rid_supreme_test_unique_hid['label'] == 1]['hid'].values
negative_hids = eeg_rid_supreme_test_unique_hid[eeg_rid_supreme_test_unique_hid['label'] == 0]['hid'].values
np.random.seed(2)

if len(negative_hids) > len(positive_hids):
    negative_hids_sampled = np.random.choice(negative_hids, size=len(positive_hids), replace=False)
else:
    negative_hids_sampled = negative_hids
test_ids_1_new = np.concatenate([positive_hids, negative_hids_sampled])
np.save('test_ids_1.npy', test_ids_1_new, allow_pickle=True)

if len(negative_hids) > len(positive_hids):
    negative_hids_sampled = np.random.choice(negative_hids, size=len(positive_hids)*2, replace=False)
else:
    negative_hids_sampled = negative_hids
test_ids_2_new = np.concatenate([positive_hids, negative_hids_sampled])
np.save('test_ids_2.npy', test_ids_2_new, allow_pickle=True)

eeg_rid_supreme_train_5 = pd.read_csv('eeg_rid_supreme_train_5.csv')

np.save('test_ids.npy', eeg_rid_supreme_test['hid'].unique(), allow_pickle=True)
np.save('train_ids_5.npy', eeg_rid_supreme_train_5['hid'].unique(), allow_pickle=True)

In [ ]:
x2= pd.read_csv('delirium_old.csv', encoding='cp949', encoding_errors='ignore')
x1= pd.read_csv('eeg_rid_supreme_sampled.csv')
sex_mapping = dict(zip(x2['연구별 환자 ID'], x2['성별']))
x1['sex'] = x1['hid'].map(sex_mapping)
x1['sex'] = x1['sex'].map({'M': 1, 'F': 0})

for idx in tqdm(range(len(x1))):
    vf = vitaldb.VitalFile('D:/combined/'+x1.loc[idx,'filename']+'.vital', track_names=['ROOT/EEG_L1', 'ROOT/EEG_L2', 'ROOT/EEG_R1', 'ROOT/EEG_R2', 'ROOT/EEG_L', 'ROOT/EEG_R','ROOT/SR'])
    vf.to_vital('D:/combined_2/'+x1.loc[idx,'filename']+'.vital')